# Disruption Analysis Compilation

#### File preparation for ArcGIS Analysis

Here we will prepare original source Vehicle Location History files for disruption analysis in ArcGIS with the workflow outlined below:

- import and slice CSV file(s) to result in dataframe with only relevant fields to analysis.
- Construct dataframe datetime fields
- Construct disruption index functions for generating a disruption index field in the dataframe.
- Export the dataframe to a new CSV file for spatial analysis in GIS.

The processes will be outlined in more detail in their own sections.

In [1]:
# Importing relevant libraries
import pandas as pd
import os
import glob
import numpy as np
import warnings
from datetime import datetime

## Import Data

File data will be imported from CSVs provided by Michael Long at Capital Metropolitan Transportation Authority. Initial data will not be provided and subsequent data will be stripped of identifiers for bus and driver identification. The only relevant data for our analysis lies in the headway time of vehicles, and time and location of record. 

In [2]:
# Setting up path vars.
path = r'../00_Source_Data/Capital_Metro/Vehicle_Location_History' # Relative source file path
all_files = glob.glob(os.path.join(path , "*.csv")) # all files

In [3]:
# This block generates a list of dataframes where each item in the list is one file.
li = []

with warnings.catch_warnings():
    warnings.simplefilter('ignore') # ignore dataframe generation warnings
    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        li.append(df)

## Construct the Dataframe

We concatenate the list of dataframes from all files in the `Vehicle_Location_History` folder into a single dataframe and slice that dataframe to contain just the information we need for analysis.

In [4]:
# This block generates a dataframe with all data from the files stored in the source file path.

frame = pd.concat(li, axis=0, ignore_index=True)

#### Formatting the Dataframe

The new dataframe we're interested in only needs the following fields:
- **timecentral**: Datetime at CST formatted as YYYYMMHHmmssss
- **nextstopid**: ID for the next stop along the route
- **lat**: Latitude the reading was taken at in WKID 4326 or WGS 1984
- **lon**: Longitude the reading was taken at in WKID 4326 or WGS 1984
- **headwaysecs**: Headway reading in seconds
- **scheduledheadwaysecs**: Planned headway in seconds

Fields that will be added later will be explained in the Disruption Index section.

In [5]:
# Var for our route of interest and the frequency we'll sample at.
## The top five routes for amount of data are the 801, 10, 803, 7, and the 20.
route = 20.0
freq = '1min'

# aggregation columns
on = 'timecentral'

In [6]:
# Saving just the fields we're interested in and removing the original data from memory.

# columns we're interested in:
cols = ['timecentral', 'nextstopid', 'lat', 'lon', 'headwaysecs', 'scheduledheadwaysecs']
li = frame.columns.to_list()

#creating a list of the columns we do not want
for col in cols:
    li.pop(li.index(col))
    
# Copying just one route's data into our dataframe
data = frame[frame['routeid'] == route]

# Dropping columns we don't want
with warnings.catch_warnings():
    warnings.simplefilter('ignore') # ignore dataframe generation warnings
    data.drop(li, axis=1, inplace=True)
del frame, li, path, all_files, df, cols

#### Converting to Datetime

Our current time field is formatted as an integer. In order to use the time values we will want to convert them into a datetime object. The current format is `20220625185013`: \
`2022` - `%Y` Year with century as a decimal \
`06`   - `%m` Month as a zero-padded decimal \
`25`   - `%d` Day of the month as a zero-padded decimal \
`18`   - `%H` Hour (24-hour clock) as a zero-padded decimal \
`50`   - `%M` Minute as a zero-padded decimal \
`13`   - `%S` Second as a zero-padded decimal

So the format to change into datetime is `%Y%m%d%H%M%S`

We can aggregate the records of the data in the requested time intervals, taking the mean location (`lat`, `lon`), mean headway values (`headwaysecs`) and the mean `disruption index` later.

In [7]:
# Lets convert the integer times into datetime objects

data['timecentral'] = pd.to_datetime(data['timecentral'], format='%Y%m%d%H%M%S')

data['timecentral'].head()

254   2022-06-19 00:00:01
255   2022-06-19 00:00:21
256   2022-06-19 00:00:51
259   2022-06-19 00:00:44
261   2022-06-19 00:00:24
Name: timecentral, dtype: datetime64[ns]

In [8]:
# Lets remove rows with NaN values for `headwaysecs` as they will not be useful for our analysis

data = data[data['headwaysecs'].notna()]
data = data[data['scheduledheadwaysecs'].notna()]

In [9]:
print(data[['headwaysecs']].max()[0],',', data[['headwaysecs']].min()[0])

5720.7 , 0.1


In [10]:
print(data[['scheduledheadwaysecs']].max()[0],',', data[['scheduledheadwaysecs']].min()[0])

1860.0 , 180.0


In [11]:
# Check to make sure it worked
print(data['headwaysecs'].isna().value_counts())
print(data['scheduledheadwaysecs'].isna().value_counts())

False    225888
Name: headwaysecs, dtype: int64
False    225888
Name: scheduledheadwaysecs, dtype: int64


## Construct Disruption Index

In this section, we will create new fields for the disruption indices and calculate them based on the work of Federico Malucelli and Emanuele Tresoldi in their case study. DOI: [s12469-019-00196-y](https://link.springer.com/article/10.1007/s12469-019-00196-y)

The fields we will construct are: \
`HR`     - Headway Ratio: the ratio between the observed headway and the planned one. \
`HS`     - Headway Standard Deviation: standard deviation of the difference between the observed and the planned headways \
`PR`     - Percentage Regularity Deviation: the percentage average ratio between the deviation of the observed headway from the planned one and the planned headway. \
`PW`     - Piece-wise linear regularity index




First, we will define a frequency with which to do our sampling. For a list of the appropriate strings to enter into the frequency variable please see [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases).

In [12]:
# Create the left side of the dataframe grouped in the chosen interval. 
# This will be concatenated with the calculated fields later and serve as the master dataframe that we'll export later.

master_1 = data[['nextstopid', 'timecentral']].resample(freq, on=on).max()
master_2 = data.groupby(pd.Grouper(key='timecentral', freq=freq)).mean()
master_1.drop(columns='timecentral', axis=1, inplace=True)
master_2.drop(columns='nextstopid', axis=1, inplace=True)

master_df = pd.concat([master_1, master_2], axis=1)
master_df

,nextstopid,headwaysecs,lat,lon,scheduledheadwaysecs
timecentral,,,,,
2022-06-19 00:00:00,5965.0,1722.966667,30.287637,-97.709945,1800.0
2022-06-19 00:01:00,5965.0,1712.375000,30.290038,-97.705720,1800.0
2022-06-19 00:02:00,2607.0,1641.275000,30.294023,-97.703555,1800.0
2022-06-19 00:03:00,1604.0,1657.050000,30.295455,-97.700085,1800.0
2022-06-19 00:04:00,5940.0,1645.600000,30.295490,-97.698025,1800.0
...,...,...,...,...,...
2022-06-25 23:55:00,4326.0,1983.950000,30.265249,-97.697847,1800.0
2022-06-25 23:56:00,5616.0,1938.822222,30.264969,-97.701893,1800.0
2022-06-25 23:57:00,6289.0,1970.611111,30.281550,-97.694847,1800.0


#### Headway Statistic Functions

Lets start by making the function that will generate the three statistical fields for our dataframe. These functions are as follows:
$$ HR=1-\bigg | \dfrac{\sum_{q \in P} \dfrac{v_o (q)}{v_e (q)}}{|P|} -1 \bigg | = 1-\bigg | \dfrac{\sum_{\text{interval}} \text{ratio}}{\text{records in interval}} -1 \bigg |\\
HS=1- \dfrac{\text{std({$v_o (q), \forall q \in P$})}}{\text{avg({$v_e (q), \forall q \in P$})}} \\
PR=1- \dfrac{\sum_{q \in P} \dfrac{|v_o (q) - v_e (q)|}{v_e (q)}}{|P|} = 1- \dfrac{\sum_{\text{interval}} \dfrac{|\text{gap}|}{v_e (q)}}{\text{records in interval}}$$

In [13]:
# Here we will do our calculations for the HR function.

# Generate intermediate calculations.
data['gap'] = data['headwaysecs'] - data['scheduledheadwaysecs']
data['ratio'] = data['headwaysecs']/data['scheduledheadwaysecs']
    
# define new dataframes
data_0 = pd.DataFrame() 
data_1 = pd.DataFrame()
data_2 = pd.DataFrame()
    
li = data.columns.to_list()
li.pop(li.index('timecentral'))

# Sum ratios by time freq
data_1 = data.resample(freq, on=on).sum()
li.pop(li.index('ratio'))
data_1.drop(li, axis=1, inplace=True)
    
# Generate a count of elements in the aggregate.
data_2 = data.resample(freq, on=on).count()
data_2.drop(li, axis=1, inplace=True)
data_2.rename(columns = {'ratio':'count'}, inplace=True)

# Calculate the headway ratio 'HR'
data_0['HR'] = 1-abs((data_1['ratio']/data_2['count'])-1)

# Now we can concatenate our HR value to the master list and remove our intermediate dataframes from memory.
master_df = pd.concat([master_df, data_0], axis=1)
del data_0, data_1, data_2, li

In [14]:
# Here we will do our calculations for the HS function.

# define new dataframes
data_0 = pd.DataFrame() 
data_1 = pd.DataFrame()
data_2 = pd.DataFrame()


li = data.columns.to_list()
li.pop(li.index('timecentral'))

# Calculate deviation within intervals of headway
data_1 = data.resample(freq, on=on).std()
li.pop(li.index('headwaysecs'))
data_1.drop(li, axis=1, inplace=True)
data_1.rename(columns={'headwaysecs':'o_std'}, inplace=True)

# refresh column list
li = data.columns.to_list()
li.pop(li.index('timecentral'))

# Calculate average of expected headways within intervals
data_2 = data.resample(freq, on=on).mean()
li.pop(li.index('scheduledheadwaysecs'))
data_2.drop(li, axis=1, inplace=True)
data_2.rename(columns={'scheduledheadwaysecs':'e_avg'}, inplace=True)

# Run the HS calculation
data_0['HS'] = 1-(data_1['o_std']/data_2['e_avg'])

# Concatenate HS values to master dataframe and delete intermediate variables from memory
master_df = pd.concat([master_df, data_0], axis=1)
del data_0, data_1, data_2, li

In [15]:
# Here we will do our calculations for the PR function.

# define new dataframes
data_0 = pd.DataFrame() 
data_1 = pd.DataFrame()
data_1_1 = pd.DataFrame()
data_2 = pd.DataFrame()

# First, we need to calculate our gap/expected-headway
data_1_1 = data
data_1_1['gap_expected'] = data_1_1['gap']/data_1_1['scheduledheadwaysecs']

li = data_1_1.columns.to_list()
li.pop(li.index('timecentral'))

# Sum the resulting gap_expected for our numerator
data_1 = data_1_1.resample(freq, on=on).sum()
li.pop(li.index('gap_expected'))
data_1.drop(li, axis=1, inplace=True)

# calculate records in interval 'P'
data_2 = data.resample(freq, on=on).count()
data_2.drop(li, axis=1, inplace=True)
data_2.rename(columns = {'gap_expected':'count'}, inplace=True)

# Calculate PR
data_0['PR'] = 1- (data_1['gap_expected']/data_2['count'])

# Concatenate HS values to master dataframe and delete intermediate variables from memory
master_df = pd.concat([master_df, data_0], axis=1)
del data_0, data_1, data_2, li, data_1_1

Lets consider the gap $x(q)$ in the planned $(v_e (q))$ and observed $(v_o (q))$ headways:
$$ x(q) = v_o (q) - v_e (q) $$

A negative value is an early pass and a positive value is a delay.

Malucelli and Tresoldi then define the function of $f(x(q))$ of the gap as:
$$
f(x(q))=
\begin{cases}
-\alpha x(q) & \quad \text{if $x(q) < -\theta_1$}\\
0 & \quad \text{if $-\theta_1 \leq x(q) < \theta_2$}\\
\beta x(q) & \quad \text{if $\theta_2 \leq x(q) < \theta_3$}\\
\gamma x(q)+\delta & \quad \text{if $\theta_3 \leq x(q)$}\\
\end{cases}
$$

Where $\theta_1$, $\theta_2$, $\theta_3$ and $\alpha, \beta, \gamma, \delta$ are suitable parameters. The function is 0 if the pass is regular and is not 0 if the pass is irregular. We can ignore contributions of earliness on the index by setting $\alpha = 0$. Likewise, if we set all coefficients to 0 and $\delta = 1$ we are left with the simple index the Azienda Trasporti Milanesi used at the writing of their paper. Values where $x(q) \geq \theta_3$ are intended to penalize large gaps more than the equivlent sum of small gaps.

Thus the index of regularity based on Malucelli and Tresoldi's piece-wise function is:
$$
I(PW)= \sum_{q \in P} f(x(q))
$$

Where $P$ is the set of all passes in a given period.

In consideration of time, we will only be constructing the piece-wise function from Malucelli and Tresoldi and using its related index for the purposes of analysis.

In [16]:
# Here we will calculate the Malucelli-Tresoldi Piecewise Index for the intervals.

# define new dataframes
data_0 = pd.DataFrame() 

# define a function for the conditions of the piecewise function.
# Our default parameters will be equal to the simplist function. Labeled as 01 by Malucelli and Tresoldi.
def piecewise(X, a=0, b=0, g=0, d=1, t_1=120, t_2=120, t_3=240):
    if X < -t_1:
        return -a*X
    elif X >= -t_1 and X < t_2:
        return 0
    elif X >= t_2 and X < t_3:
        return b*X
    elif X >= t_3:
        return g*X+d

# apply our piecewise function
li = []
for row in data.itertuples():
    gap = row.gap
    pw = piecewise(gap)
    li.append(pw)
data['PW'] = li

# Create a list of columns for data_1 to drop later.
li = data.columns.to_list()
li.pop(li.index('timecentral'))

# Sum our 'PW' values by our intervals to get our index
data_0 = data.resample(freq, on=on).sum()
li.pop(li.index('PW')) # We want to keep the 'PW' column
data_0.drop(li, axis=1, inplace=True)
data_0.rename(columns={'PW':'I_PW'}, inplace=True)

# Concatenate I_PW values to master dataframe and delete intermediate variables from memory
master_df = pd.concat([master_df, data_0], axis=1)
del data_0, li

In [17]:
master_df

,nextstopid,headwaysecs,lat,lon,scheduledheadwaysecs,HR,HS,PR,I_PW
timecentral,,,,,,,,,
2022-06-19 00:00:00,5965.0,1722.966667,30.287637,-97.709945,1800.0,0.957204,0.941931,1.042796,0.0
2022-06-19 00:01:00,5965.0,1712.375000,30.290038,-97.705720,1800.0,0.951319,0.964636,1.048681,0.0
2022-06-19 00:02:00,2607.0,1641.275000,30.294023,-97.703555,1800.0,0.911819,0.957788,1.088181,0.0
2022-06-19 00:03:00,1604.0,1657.050000,30.295455,-97.700085,1800.0,0.920583,0.938050,1.079417,0.0
2022-06-19 00:04:00,5940.0,1645.600000,30.295490,-97.698025,1800.0,0.914222,0.948184,1.085778,0.0
...,...,...,...,...,...,...,...,...,...
2022-06-25 23:55:00,4326.0,1983.950000,30.265249,-97.697847,1800.0,0.897806,0.529081,0.897806,3.0
2022-06-25 23:56:00,5616.0,1938.822222,30.264969,-97.701893,1800.0,0.922877,0.549613,0.922877,2.0
2022-06-25 23:57:00,6289.0,1970.611111,30.281550,-97.694847,1800.0,0.905216,0.444576,0.905216,3.0


## Export Dataframe

The last thing to do is export the dataframe for spatial analysis in ArcGIS Pro.

In [18]:
master_df.to_csv("../01_New_Data/Python_Output/Route_"+str(route)+"_Disruption.csv")